# Импорт и установка всего


In [1]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 27.2 MB/s 


In [2]:
!pip install summa

     |████████████████████████████████| 54 kB 2.2 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=5a28439ec4beb8c6c62a8b84df0aff04df4e66170332763b5167003f7bb3a41b
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [3]:
!pip install python-rake

In [4]:
!pip install yargy

     |████████████████████████████████| 41 kB 101 kB/s 


In [5]:
import pymorphy2
import nltk
import numpy as np
import RAKE
import pandas as pd


from string import digits
from yargy import Parser
from yargy.pipelines import morph_pipeline
from pymorphy2.tokenizers import simple_word_tokenize
from yargy import rule, or_
from yargy.interpretation import fact
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
from yargy import Parser, rule, and_
from yargy.predicates import gram, dictionary


nltk.download('stopwords')
stopword = stopwords.words('russian')
stopword.append('это')
stopword.append('который')
regexp_tokenizer = RegexpTokenizer(r'\w+')
pymorph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Основная часть

Чтение датафрейма

In [6]:
df = pd.read_csv('corpus_tags.csv', delimiter=';')

Устройство датафрейма (csv-файл лежит в папке с тетрадкой):


*   Text -- полный текст новости
*   Source -- ссылка текст (все тексты взяты с РИА Новости)
*   Original Tags -- ключевые слова с сайта
*   My Tags -- мои ключевые слова
*   Gold Tags -- эталон. За эталон я брал пересечение моих ключевых слов и оригинальных. Иногда в оригинальных ключевых словах можно встретить фамилию автора текста или жанр, например, "Новости" или "В мире". Такие ключевые слова я выкидывал из эталона


In [7]:
df

,Text,Source,Original Tags,My Tags,Gold Tags
0,В Дании задержали российское судно В датском п...,https://ria.ru/20211104/daniya-1757704473.html,"Дания, В мире, Россия","Дания, Россия, Академик Иоффе","Дания, Россия, Академик Иоффе"
1,Ученые сообщили о крупнейшей за несколько лет ...,https://ria.ru/20211104/burya-1757695232.html,"Наука, Физический Институт РАН, Российская ака...","Земля, Магнитные бури, Ученые, Магнитное поле","Земля, Магнитные бури, Ученые, Магнитное поле"
2,Одно страшнее другого. Установлена связь COVID...,https://ria.ru/20211102/kovid-1757233335.html,"Наука, Здоровье, Биология, болезнь Альцгеймера...","Коронавирус, деменция, COVID-19, болезнь Альцг...","Коронавирус,COVID-19, болезнь Альцгеймера, исс..."
3,Миллиардеры объединились против власти. Они не...,https://ria.ru/20211102/dengi-1757271767.html,"США, Марк Цукерберг, Джо Байден, Франклин Рузв...","США, Марк Цукерберг, Джо Байден, Amazon, Apple...","США, Марк Цукерберг, Джо Байден, Amazon, Apple..."
4,ВОЗ спрогнозировала еще полмиллиона смертей от...,https://ria.ru/20211104/koronavirus-1757707409...,"Распространение коронавируса, В мире, ВОЗ, Здо...","ВОЗ, COVID-19, Клюге","ВОЗ, COVID-19, Клюге"
5,EMA прокомментировало информацию о нарушениях ...,https://ria.ru/20211104/vaktsina-1757697051.html,"Распросранение коронавируса, В Мире, Вакцина P...","Вакцина, EMA, BMJ, Pfizer, испытания","Вакцина, EMA, BMJ, Pfizer, испытания, США"


Функции для препроца текста и извлечения ключевых слов тремя способами, извлечения эталона из корпуса

In [8]:
def preprocessing(text, tokenizer_method=simple_word_tokenize):
    lemmas = []

    for t in tokenizer_method(text):
      lemma = pymorph.parse(t)[0].normal_form
      lemmas.append(lemma)
    
    return ' '.join(lemmas)


def get_rake_tags(normalized_text, n):
    tags = []
    rake = RAKE.Rake(stopword)
    tag_list = rake.run(normalized_text, maxWords=2, minFrequency=2)

    for tag in tag_list:
      if tag[1] > 0:
        tags.append(tag[0])
    
    return tags[:n]


def get_textrank_tags(normalized_text, n):
  tag_list = keywords.keywords(normalized_text, language='russian', 
                           additional_stopwords=stopword)
  return tag_list.split('\n')[:n]


### "Это для tfidf тэггинга" ###
def sort_coo(coo_matrix):
  tuples = zip(coo_matrix.col, coo_matrix.data)
  return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)


def extract_topn_from_vector(feature_names, sorted_items, topn=10):
  sorted_items = sorted_items[:topn]
  score_vals = []
  feature_vals = []

  for idx, score in sorted_items:
      score_vals.append(round(score, 3))
      feature_vals.append(feature_names[idx])

  results= {}

  for idx in range(len(feature_vals)):
      results[feature_vals[idx]]=score_vals[idx]

  return results


def get_tfidf_tags(vectorizer, feature_names, normalized_text, n):
  tf_idf_vector = vectorizer.transform([normalized_text])
  sorted_items=sort_coo(tf_idf_vector.tocoo())
  tags=extract_topn_from_vector(feature_names,sorted_items, n)
  return list(tags.keys())
### "Конец" ###


def get_tags_docs(frame, tagging_method, n, column='Preprocessed'):
  tags = []

  if tagging_method == get_tfidf_tags:
    vectorizer = TfidfVectorizer(stop_words=stopword, smooth_idf=True, 
                             use_idf=True, ngram_range=(1,3))
    vectorizer.fit_transform(frame['Text'])
    feature_names = vectorizer.get_feature_names()

  for doc in frame[column]:
    text = doc
    if tagging_method != get_tfidf_tags:
      tag_set = tagging_method(text, n)
    else:
      tag_set = tagging_method(vectorizer, feature_names, text, n)
    
    tags.append(tag_set)
  
  return tags


def get_gold_tags(frame, column='Gold Tags'):
  tags = []

  for i in range(0, len(frame)):
    tags_set = preprocessing(frame[column][i])
    tags_set = [tag.strip() for tag in tags_set.split(',')]
    tags_set = [tag.replace('covid-19', 'covid19') for tag in tags_set]
    tags.append(tags_set)
 
  return tags


def get_metrics(predicted_tags, gold_tags):
  preсisions = []
  recalls = []
  f_scores = []

  for id, tag_set in enumerate(gold_tags):
    doc = gold_tags[id]
    TP = 0
    
    for i in range(0, len(predicted_tags[id])):
      if predicted_tags[id][i] in doc:
        TP+=1
    
    precision = TP/len(predicted_tags[id])
    preсisions.append(precision)
    recall = TP/len(doc)
    recalls.append(recall)
    if precision != 0 and recall != 0:
      f_score = 2*precision*recall/(precision+recall)
    else:
      f_score = 0
    f_scores.append(f_score)
  return preсisions, recalls, f_scores

Препроц каждого текста в корпусе. Замена covid-19 на covid19, чтобы TfIdf-векторайзер не считал это за разные слова (он так делает)

In [9]:
df['Preprocessed'] = df.apply(lambda row: preprocessing(row['Text']), axis=1)
df['Preprocessed'] = df.apply(lambda row: 
                              row['Preprocessed'].replace('covid-19', 'covid19'), 
                              axis=1)

In [10]:
gold_tags = get_gold_tags(df)

In [11]:
gold_tags

[['дания', 'россия', 'академик иофф'],
 ['земля', 'магнитный буря', 'учёный', 'магнитный поле'],
 ['коронавирус',
  'covid19',
  'болезнь альцгеймер',
  'исследование',
  'учёный',
  'деменция'],
 ['сша',
  'марк цукерберг',
  'джо байден',
  'amazon',
  'apple',
  'microsoft',
  'дональд трамп',
  'капитолий',
  'илона маск',
  'налог',
  'закон'],
 ['воз', 'covid19', 'клюг'],
 ['вакцина', 'ema', 'bmj', 'pfizer', 'испытание', 'сша']]

In [12]:
vectorizer = TfidfVectorizer(stop_words=stopword, smooth_idf=True, 
                             use_idf=True, ngram_range=(1,2), min_df=0.2,
                             max_df=0.8)
vectorizer.fit_transform(df['Text'])
feature_names = vectorizer.get_feature_names()

In [13]:
rake_tags = get_tags_docs(df, get_rake_tags, 15)
textrank_tags = get_tags_docs(df, get_textrank_tags, 15)
tfidf_tags = get_tags_docs(df, get_tfidf_tags, 15)

In [14]:
rake_tags

[['академик иофф'],
 ['сохраняться ещё', 'земля', 'фиан', 'крупный'],
 ['контрольный группа',
  'болезнь альцгеймер',
  'концентрация внимание',
  'учёный',
  'человек',
  'память',
  'мышление',
  'covid19',
  'исследование',
  'причём',
  'ивл'],
 ['список forbes',
  'дональд трамп',
  'налог',
  'трамп',
  'инвестиция',
  'пока',
  'доход',
  'взять',
  'будущее',
  'миллиардер',
  'продать',
  'прибыль',
  'марс',
  'twitter',
  'попасть'],
 ['covid19', 'европа', 'человек'],
 ['сомнение вывод', 'безопасность', 'нарушение', 'ставить', 'эффективность']]

In [15]:
textrank_tags

[['судный',
  'задержать российский судно',
  'академик иофф',
  'научно',
  'научный',
  'посольство',
  'обеспечительный',
  'распространение',
  'третий сторона',
  'инцидент произойти',
  'датский порт скаген',
  'советский'],
 ['год магнитная буре',
  'магнитный буря',
  'солнечный',
  'учёный',
  'вспышка',
  'земля',
  'около час',
  'сияние',
  'зона',
  'последний',
  'северный',
  'сутки',
  'межпланетный',
  'шкала',
  'фиан'],
 ['covid',
  'мозг',
  'деменции',
  'деменция',
  'болезнь альцгеймер',
  'тест',
  'иммунный',
  'связь',
  'учёный',
  'исследование',
  'ген',
  'поражать человек',
  'клетка',
  'больший',
  'тяжёлый'],
 ['байден',
  'налог',
  'налоговый',
  'весь',
  'сенатор',
  'доход',
  'корпорация',
  'законопроект мочь',
  'демократ левый',
  'компания',
  'около',
  'представитель',
  'сми',
  'большой миллиард',
  'процент'],
 ['воз',
  'полмиллиона',
  'случай',
  'время',
  'миллион',
  'европейский регион',
  'мера',
  'организация',
  'год',
  'ситу

In [16]:
tfidf_tags

[['судно',
  'академик',
  'российский',
  'судно академик',
  'скаген',
  'россия',
  'посольство',
  'научный',
  'научно',
  'использовать',
  'институт',
  'честь',
  'физика',
  'статус',
  'сотрудник'],
 ['буря',
  'уровень',
  'магнитная',
  'фиан',
  'зона',
  'буре',
  'несколько',
  'лишь',
  'четыре',
  'сохраняться',
  'россия',
  'прогноз',
  'поле',
  'нестабильность',
  'мск'],
 ['болезнь',
  'мозг',
  'связь',
  'человек',
  'тест',
  'коронавирус',
  'когнитивный',
  'симптом',
  'исследование',
  'иммунный',
  'деменции',
  'ген',
  'несколько',
  'риск',
  'память'],
 ['налог',
  'миллиард',
  'байден',
  'трамп',
  'сенатор',
  'платить',
  'законопроект',
  'доллар',
  'страна',
  'реформа',
  'процент',
  'президент',
  'план',
  'маск',
  'компания'],
 ['полмиллиона',
  'воз',
  '2022',
  'человек',
  'страна',
  'спасти',
  'ситуация',
  'данные',
  'организация',
  'время',
  'человек полмиллиона',
  'ханс',
  'умереть',
  'ужесточить',
  'старший'],
 ['ema',
 

Посчитаем средние скоры для каждого метода извлечения

In [17]:
def get_mean_scores(tags, gold_tags):
  tags_scores = get_metrics(tags, gold_tags)
  tags_precision = np.mean(tags_scores[0])
  tags_recall = np.mean(tags_scores[1])
  tags_f_scores = np.mean(tags_scores[2])
  
  return 'precision: {}, recall: {}, f-score: {}'.format(
      tags_precision, tags_recall, tags_f_scores)

In [18]:
get_mean_scores(rake_tags, gold_tags)

'precision: 0.34671717171717176, recall: 0.29419191919191917, f-score: 0.2846279537456008'

In [19]:
get_mean_scores(textrank_tags, gold_tags)

'precision: 0.125, recall: 0.39015151515151514, f-score: 0.18555791187370133'

In [20]:
get_mean_scores(tfidf_tags, gold_tags)

'precision: 0.1111111111111111, recall: 0.3207070707070707, f-score: 0.16096866096866094'

В эталонах часто встречаются: сочетания существительное + существительное, прилагательное + существительное, просто существительное и имена. Еще в эталонных ключевых словах можно найти имена собственные на латиница: названия корпораций, вируса или вакцин, например, Pfizer и COVID-19. Напишем грамматику и извлечем только ключевые слова, которые удовлетворяют этим критериям.

In [21]:
# Функция для извлечения всех эталонных ключевых слов на латнице
def extract_latin(gold_tag):
  latin = []
  for tag in gold_tags:
    for t in tag:
      if t.isascii():
        latin.append(t)
  return latin

In [78]:
Name = fact(
    'Name',
    ['first', 'last']
)


NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

BRANDS = morph_pipeline(extract_latin(gold_tags))


AdjP = rule(gram('ADJF'), gram('NOUN'))
NP = rule(gram('NOUN'), gram('NOUN'))
N = rule(gram('NOUN'))

grammar = Parser(
    or_(
        NAME,
        AdjP,
        NP,
        N,
        BRANDS
    )
)

Извлечем ключвые слова, которые соответствуют написанным нами правилам

In [73]:
def template_tags(tag_set):
  new_tags = []
  for tag in tag_set:
    for match in grammar.findall(tag):
      new_tag = ' '.join([_.value for _ in match.tokens])
      new_tag = new_tag.replace('covid 19', 'covid19')
      new_tags.append(new_tag)
  return new_tags

In [74]:
def get_new_tag_sets(tags):
  new_tag_sets = []
  for tag in tags:
    new_tag_sets.append(template_tags(tag))
  return new_tag_sets

Посмотрим на метрики

In [69]:
new_rake = get_new_tag_sets(rake_tags)
new_textrank = get_new_tag_sets(textrank_tags)
new_tfidf = get_new_tag_sets(tfidf_tags)

In [75]:
new_rake

[['академик иофф'],
 ['земля', 'фиан'],
 ['контрольный группа',
  'болезнь альцгеймер',
  'концентрация внимание',
  'учёный',
  'человек',
  'память',
  'мышление',
  'covid19',
  'исследование'],
 ['список',
  'дональд трамп',
  'налог',
  'трамп',
  'инвестиция',
  'доход',
  'будущее',
  'миллиардер',
  'прибыль',
  'марс'],
 ['covid19', 'европа', 'человек'],
 ['сомнение вывод', 'безопасность', 'нарушение', 'эффективность']]

In [84]:
get_mean_scores(new_rake, gold_tags)

'precision: 0.412962962962963, recall: 0.29419191919191917, f-score: 0.31507936507936507'

In [85]:
get_mean_scores(new_textrank, gold_tags)

'precision: 0.21042152292152294, recall: 0.47348484848484845, f-score: 0.28577688651218064'

In [86]:
get_mean_scores(new_tfidf, gold_tags)

'precision: 0.15331890331890333, recall: 0.34848484848484845, f-score: 0.2091005291005291'

Как можно видеть, правильно выделенные шаблоны увеличивают метрики. Главное, чтобы в правила и шаблоны попадали все ключевые слова из эталонов. Увеличение метрик (точности и f-score) происходит же в основном из-за сокращения числа предложенных ключевых слов, так как те слова, которые не подходят шаблонам не берутся в расчет.

Кроме того, если применить фильтры, то из строк вроде "дональд трамп продать" будет выделяться то, что соответствует грамматике, например, имя "дональд трамп", что увеличивает количество правильно предсказанных ключевых слов.

# Что можно улучшить

В-первую очередь, мне кажется разумным было бы как-то унифицировать имена. Например, можно выделить в качестве ключевого слова "Джо Байден", а извлекать программа будет ключевое слово "Байден". С одной стороны, это одно и то же, но для того, чтобы это отразилось на метриках, нужно чтобы слова полностью совпадали. Разумным при выделении имен было бы оставлять только фамилию, как в эталонных ключевых словах, так и в извлеченных.

Еще можно было бы как-то конкретизировать существительные для шаблонов. Не просто выделять существительное, но на более большом корпусе текстов посмотреть на то какие существительные по-одиночку встречаются в эталонах. Например, одной из конкретизаций может быть выделение названий стран. Я не уверен, что это хорошая идея, потому что проще все таки сказать "выделяй существительные по-одиночке" чем перечислить какие они могут быть.

В этой тетрадке я вручную ввожу максимульную длину ключевых слов (в данном случае 2). Можно было бы автоматически считать самое длинное ключевое слово в эталоне и предсказывать ключевые слова максимум его длины.

P.S. Я также хотел бы немного высказаться по формулировке задания. Мне кажется важно сначала именно самому выделять в тексте ключевые слова, а потом смотреть как они в нем уже выделены. В противном случае как будто создается bias. Я, например, заметил, что выделяю почти те же ключевые слова и возможно, если бы я не видел оригинальные до этого, я бы сделал все чуть-чуть по-другому.

